In [1]:
# DELAYED COLUMN GENERATION

In [ ]:
Optimal cutting using known patterns

In [1]:
stocks = {
    "A": {"length": 5, "cost": 6},
    "B": {"length": 6, "cost": 7},
    "C": {"length": 9, "cost": 10},
}

finish = {
    "S": {"length": 2, "demand": 20},
    "M": {"length": 3, "demand": 10},
    "L": {"length": 4, "demand": 20},
}

In [2]:
def make_naive_patterns(stocks, finish):
    """
    Generates patterns of feasible cuts from stock lengths to meet specified finish lengths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts = int(stocks[s]["length"] / finish[f]["length"])

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for {f}")
            return []

    return patterns


patterns = make_naive_patterns(stocks, finish)
display(patterns)

[{'stock': 'A', 'cuts': {'S': 2, 'M': 0, 'L': 0}},
 {'stock': 'B', 'cuts': {'S': 3, 'M': 0, 'L': 0}},
 {'stock': 'C', 'cuts': {'S': 4, 'M': 0, 'L': 0}},
 {'stock': 'A', 'cuts': {'S': 0, 'M': 1, 'L': 0}},
 {'stock': 'B', 'cuts': {'S': 0, 'M': 2, 'L': 0}},
 {'stock': 'C', 'cuts': {'S': 0, 'M': 3, 'L': 0}},
 {'stock': 'A', 'cuts': {'S': 0, 'M': 0, 'L': 1}},
 {'stock': 'B', 'cuts': {'S': 0, 'M': 0, 'L': 1}},
 {'stock': 'C', 'cuts': {'S': 0, 'M': 0, 'L': 2}}]

In [3]:
import cvxpy as cp

def cut_patterns_cv(stocks, finish, patterns):
    # Define sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    c = [stocks[patterns[p]["stock"]]["cost"] for p in P]
    a = {(f, p): patterns[p]["cuts"].get(f, 0) for p in P for f in F}
    demand_finish = {f: finish[f]["demand"] for f in F}

    # Variables
    x = cp.Variable(len(P), integer=True, nonneg=True)

    # Objective function: minimize cost
    cost = cp.sum([c[p] * x[p] for p in P])

    # Constraints
    constraints = []
    for f in F:
        constraints.append(cp.sum([a[f, p] * x[p] for p in P]) >= demand_finish[f])

    # Formulate the problem
    problem = cp.Problem(cp.Minimize(cost), constraints)

    # Solve the problem
    problem.solve(solver=cp.CPLEX)

    return x.value, problem.value

# Example usage:
x, cost = cut_patterns_cv(stocks, finish, patterns)
print(f"Optimal pattern choice: {x}")
print(f"Minimum cost: {cost}")

ModuleNotFoundError: No module named 'cvxpy'

In [4]:
from ortools.linear_solver import pywraplp

def cut_patterns_ortools(stocks, finish, patterns):
    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('Solver not created.')

    # Define sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    c = [stocks[patterns[p]["stock"]]["cost"] for p in P]
    a = {(f, p): patterns[p]["cuts"].get(f, 0) for p in P for f in F}
    demand_finish = {f: finish[f]["demand"] for f in F}

    # Variables
    x = [solver.IntVar(0, solver.infinity(), f'x[{p}]') for p in P]

    # Objective function: minimize cost
    objective = solver.Objective()
    for p in P:
        objective.SetCoefficient(x[p], c[p])
    objective.SetMinimization()

    # Constraints
    for f in F:
        constraint = solver.Constraint(demand_finish[f], solver.infinity())
        for p in P:
            constraint.SetCoefficient(x[p], a[f, p])

    # Solve the problem
    status = solver.Solve()

    if status != pywraplp.Solver.OPTIMAL:
        raise Exception('The problem does not have an optimal solution.')

    # Get the results
    x_values = [x[p].solution_value() for p in P]
    cost = solver.Objective().Value()

    return x_values, cost

# Example usage:
x, cost = cut_patterns_ortools(stocks, finish, patterns)
print(f"Optimal pattern choice: {x}")
print(f"Minimum cost: {cost}")


Optimal pattern choice: [1.0, 6.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 10.0]
Minimum cost: 182.0
